In [ ]:
import random
import heapq

# Define the problem - a list of cities and their distances
cities = {
    'Madrid': {'Barcelona': (98,150), 'Paris': (380,225) },
    'Barcelona': {'Madrid': (98,150), 'Lyon': (320,200), 'Paris': (400,390)},
    'Lyon': {'Barcelona': (320,200), 'Milan': (180,176), 'Paris': (185,112)},
    'Milan': {'Lyon': (180,176), 'Rome': (125,168), 'Frankfurt': (240,454)},
    'Rome': {'Milan': (125,168)},
    'Frankfurt': {'Milan': (240,454), 'Paris': (345,480), 'Cologne': (40,120), 'Berlin': (125,232)},
    'Berlin': {'Frankfurt': (2125,232), 'Amsterdam': (235,364)},
    'Amsterdam': {'Berlin': (235,364), 'Cologne': (40,120), 'Brussels': (48,105)},
    'Cologne': {'Amsterdam': (40,120), 'Frankfurt': (40,120)},
    'Paris': {'Madrid': (380,225), 'Barcelona': (400,390), 'Lyon': (185,112), 'Frankfurt': (345,480), 'Brussels': (80,82), 'London': (98,136)},
    'London': {'Paris': (98,136), 'Brussels': (98,136)},
    'Brussels': {'London': (198,136), 'Paris': (80,82), 'Amsterdam': (48,105)},
}

# Define the parameters of the genetic algorithm
population_size = 2
num_generations = 0
mutation_rate = 0.1


# Dijkstra's algorithm for finding the shortest path in a graph
def dijkstra(graph, start, end):
    heap = [(0, start, [])]
    visited = set()
    while heap:
        (cost, curr, path) = heapq.heappop(heap)
        if curr not in visited:
            visited.add(curr)
            path = path + [curr]
            if curr == end:
                return path
            for neighbor, weight in graph[curr].items():
                if neighbor not in visited:
                    heapq.heappush(heap, (cost + weight[0], neighbor, path))
    return None

# Helper function for make_viable()
def remove_adjacent(nums):
     return [a for a,b in zip(nums, nums[1:]+[not nums[-1]]) if a != b]

# Checks if possible solution is a good solution. If not, it converts it so.
def make_viable(graph, psol):
    sol = []
    temp = []
    i = 0
    while i < len(psol)-1:
        if psol[i+1] not in graph.get(psol[i]):
            temp = dijkstra(cities, psol[i], psol[i+1])
            sol.extend(temp)
        else:
            sol.append(psol[i])
        temp = []
        i = i+1
        sol = remove_adjacent(sol)
    return sol

# Initialize the population of potential solutions
def create_individual():
    cities_list = list(cities.keys())
    random.shuffle(cities_list)
    for i in range(len(cities_list)):
        cities_list = make_viable(cities, cities_list)
    return cities_list

population = [create_individual() for _ in range(population_size)]

# Define the fitness function
def calculate_fitness(individual):
    # Calculate the total distance of the route
    distance = 0
    for i in range(len(individual)-1):
        current_city = individual[i]
        next_city = individual[(i+1) % len(individual)]
        distance += cities[current_city][next_city][1]
        if distance > 4320:
            return 0
    # Return the inverse of the distance as the fitness
    return 1 / distance

# Define the selection function
def selection(population):
    # Use tournament selection to choose parents
    tournament_size = 2
    parents = []
    for _ in range(len(population)):
        tournament = random.sample(population, tournament_size)
        winner = max(tournament, key=calculate_fitness)
        parents.append(winner)
    return parents

# Define the crossover function
def crossover(parent1, parent2):
    # Choose a random crossover point
    crossover_point = random.randint(1, len(parent1)-1)
    # Create the offspring by swapping the tails of the parents
    offspring1 = parent1[:crossover_point] + [city for city in parent2 if city not in parent1[:crossover_point]]
    offspring2 = parent2[:crossover_point] + [city for city in parent1 if city not in parent2[:crossover_point]]
    offspring1 = make_viable(cities, offspring1)
    offspring2 = make_viable(cities, offspring2)
    return offspring1, offspring2

# Define the mutation function
def mutation(individual, mutation_rate):
    # Swap two cities with a given probability
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            j = random.randint(0, len(individual)-1)
            individual[i], individual[j] = individual[j], individual[i]
    individual = make_viable(cities, individual)

# Define the replacement function
def replacement(population, offspring):
    # Replace the least fit individuals with the offspring
    population = sorted(population, key=calculate_fitness)
    num_to_replace = len(offspring)
    population[-num_to_replace:] = offspring
    for i in range(len(population)):
        population[i] = make_viable(cities, population[i]) 
    return population

# Run the genetic algorithm
for generation in range(num_generations):
    print(f'Generation: {generation}')
    # Evaluate the fitness of the population
    fitness_scores = [calculate_fitness(individual) for individual in population]
    # Select parents and create offspring
    parents = selection(population)
    offspring = []
    for i in range(0, len(parents), 2):
        offspring1, offspring2 = crossover(parents[i], parents[i+1])
        mutation(offspring1, mutation_rate)
        mutation(offspring2, mutation_rate)
        offspring.extend([offspring1, offspring2])
    # Replace the least fit individuals with the offspring
    print()
    population = replacement(population, offspring)
    # Make population travel to ALL the cities, then make viable.

# Print the best solution found
best_individual = max(population, key=calculate_fitness)
best_fitness = calculate_fitness(best_individual)
print(f'Best individual: {best_individual}')
print(f'Best fitness: {best_fitness:.2f}')

    

